In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # lib to transform text in chunks
from langchain_openai import OpenAIEmbeddings # lib for the embedding model
from langchain_openai import ChatOpenAI 
from langchain_community.vectorstores import Chroma # vector database
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain # lib to make prompt

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
# Load of models (Embeddings and LLM)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens = 200)

In [13]:
# Load PDF

pdf_link = "CSJoseph_Grid_for_Typing_Anyone_LAYOUT_V21.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [14]:
# Separate in chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000, # token size
    chunk_overlap = 20, # mantain chunk consistent
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [16]:
# Save on Vector DB - Chroma

database = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
database.persist()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [17]:
# Load DB
vectorDb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

# Load Retriever
retriever = vectorDb.as_retriever(search_kwargs={"k":3})

# Construction of chain prompt to call LLM
chain = load_qa_chain(llm, chain_type="stuff")

C:\Users\arthu\AppData\Local\Temp\ipykernel_33096\300528918.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorDb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)
C:\Users\arthu\AppData\Local\Temp\ipykernel_33096\300528918.py:8: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/mi

In [18]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_output=True))['output_text']
    return answer

In [19]:
user_question = input("User: ")
answer = ask(user_question)
print("Answer: ", answer)

User:  O que o documento diz sobre as personalidades terminadas em NJ


C:\Users\arthu\AppData\Local\Temp\ipykernel_33096\773938753.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(question)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}